In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

import folium
import pandas as pd
from folium.plugins import MarkerCluster, MousePosition
from folium.features import DivIcon
from math import sin, cos, sqrt, atan2, radians
import io
from js import fetch

In [2]:
#load launch site data
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df = pd.read_csv(spacex_csv_file)

In [3]:
#select relevant columns
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()[['Launch Site', 'Lat', 'Long']]

In [4]:
#mark all launch sites on map
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# # of launches per site
launch_counts = spacex_df['Launch Site'].value_counts().to_dict()

#mark each site
for index, row in launch_sites_df.iterrows():
    site = row['Launch Site']
    coord = [row['Lat'], row['Long']]
    radius = launch_counts.get(site, 1) * 100  # Dynamic radius

    folium.Circle(coord, radius=radius, color='#d35400', fill=True, fill_color='#e67e22').add_child(
        folium.Popup(site)
    ).add_to(site_map)

    folium.Marker(coord,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{site}</b></div>'
        )
    ).add_to(site_map)

site_map

In [5]:
#mark success/failure
marker_cluster = MarkerCluster()

#assign marker colors
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

for index, row in spacex_df.iterrows():
    marker = folium.Marker([row['Lat'], row['Long']],
                icon=folium.Icon(color='white', icon_color=row['marker_color']))
    marker_cluster.add_child(marker)

site_map.add_child(marker_cluster)

In [6]:
#proximity markers and distance lines
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter
)
site_map.add_child(mouse_position)

#calculate haversine dist.
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

In [7]:
#draw line from CCAFS LC-40 to a nearby coastline point
launch_site_coord = [28.562302, -80.577356]  # CCAFS LC-40
coastline_coord = [28.56402, -80.56811]       # manually identified nearby point

#calculate and mark distance
distance = calculate_distance(*launch_site_coord, *coastline_coord)
folium.Marker(coastline_coord,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12; color:#d35400;"><b>{distance:.2f} KM</b></div>'
    )
).add_to(site_map)

#draw polyline
lines = folium.PolyLine(locations=[launch_site_coord, coastline_coord], weight=2)
site_map.add_child(lines)